<a href="https://colab.research.google.com/github/robd123/AML/blob/main/Project3/Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
import os
datapath = os.path.join("datasets", "heart", "")
os.makedirs(datapath, exist_ok=True)
file_path = "https://raw.githubusercontent.com/robd123/AML/main/Project3/heart.csv"
urllib.request.urlretrieve(file_path, datapath + "heart.csv")

('datasets/epi/epi_r.csv', <http.client.HTTPMessage at 0x7fbcf24131d0>)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

heart_data = pd.read_csv(datapath + "heart.csv", thousands=',')